# X-ray photoelectron spectroscopy (XPS)
For the source code, see [xps](https://workgraph-collections.readthedocs.io/en/latest/qe/module.html#workgraph_collections.ase.espresso.xps.xps_workgraph).


## Visualizing the WorkGraph Builder


## Example: ETFA molecule

### Prepare the inputs and submit the workflow


In [1]:
from ase.io import read
from aiida import load_profile
from aiida.orm import (
    Dict,
    KpointsData,
    StructureData,
    load_code,
    load_group,
    QueryBuilder,
    Group,
)
from workgraph_collections.qe.xps import XpsWorkgraph

load_profile()

def load_core_hole_pseudos(pseudo_group="pseudo_demo_pbe"):
    """Load the core hole pseudos."""
    pseudo_group = (
        QueryBuilder().append(Group, filters={"label": pseudo_group}).one()[0]
    )
    pseudos = {"core_hole": {
        pseudo.label.split("_")[0]: pseudo for pseudo in pseudo_group.nodes if "gs" not in pseudo.label
    },
    "gipaw": {
        pseudo.label.split("_")[0]: pseudo for pseudo in pseudo_group.nodes if "gs" in pseudo.label
    }}
    return pseudos, pseudo_group.base.extras.get("correction", {})


# create input structure
etfa = read("datas/ETFA.xyz")
structure = StructureData(ase=etfa)
# create the PW node
code = load_code("pw-7.4@localhost")
# code = load_code("pw-7.4@eiger")
parameters = Dict(
    {
        "CONTROL": {
            "calculation": "scf",
        },
        "SYSTEM": {
            "ecutwfc": 40,
            "ecutrho": 300,
            "occupations": "fixed",
        },
    }
)
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
# Load the pseudopotential family.
core_hole_pseudos, correction_energies = load_core_hole_pseudos("pseudo_demo_pbe")
correction_energies = {
    key.split("_")[0]: value["core"] for key, value in correction_energies.items()
}
pseudo_family = load_group("SSSP/1.3/PBE/efficiency")
pseudos = pseudo_family.get_pseudos(structure=structure)
#
metadata = {
    "options": {
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 4,
        },
    }
}
metadata_eiger = {
    "options": {
        "custom_scheduler_commands": "#SBATCH --account=mr32",
        "resources": {
            "num_machines": 1,
            "num_mpiprocs_per_machine": 36,
        },
    }
}
# ===============================================================================
wg = XpsWorkgraph.build(
    structure=structure,
    code=code,
    # atoms_list=[0, 1, 2, 3],
    element_list=["C"],
    parameters=parameters,
    kpoints=kpoints,
    pseudos=pseudos,
    is_molecule=True,
    core_hole_pseudos=core_hole_pseudos,
    correction_energies=correction_energies,
    metadata=metadata,
)
wg.to_html()

Run the workgraph

In [ ]:
wg.run()

09/02/2025 03:12:25 PM <1488790> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [175827|WorkGraphEngine|continue_workgraph]: tasks ready to run: get_xspectra_structures
09/02/2025 03:12:26 PM <1488790> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [175827|WorkGraphEngine|update_task_state]: Task: get_xspectra_structures, type: CALCFUNCTION, finished.
09/02/2025 03:12:26 PM <1488790> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [175827|WorkGraphEngine|continue_workgraph]: tasks ready to run: run_scf
09/02/2025 03:12:31 PM <1488790> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [175827|WorkGraphEngine|on_wait]: Process status: Waiting for child processes: 175849
09/02/2025 03:12:36 PM <1488790> aiida.orm.nodes.process.workflow.workchain.WorkChainNode: [REPORT] [175849|WorkGraphEngine|continue_workgraph]: tasks ready to run: PwCalculation,PwCalculation1,PwCalculation2,PwCalculation3,PwCalculation4
09/02/20

result: 
  site_0: 305.1129763777
  site_1: 302.0445550643
  site_2: 299.7042622079
  site_3: 297.9095340518


Print the results:

In [3]:
print("result: ")

for key, value in wg.outputs.result.value.value.items():
    print(f"  {key}: {value}")

result: 
  site_0: 305.1129763777
  site_1: 302.0445550643
  site_2: 299.7042622079
  site_3: 297.9095340518
